# Phylogenetic comparative methods

### using [BayesTraits](http://www.evolution.rdg.ac.uk/BayesTraits.html) 3.0.1
#### Motif set: (Sp-D, Sp-L, Maze)
#### Top 10 major fish orders


In [ ]:
version

In [ ]:
library(tidyverse)
library(ape)
library(phytools)
library(fishtree)

packageVersion("tidyverse")
packageVersion("ape")
packageVersion("phytools")
packageVersion("fishtree")

In [ ]:
# num_gen: the number of genera
# num_gen2: the number of genera for which trait data are available
# num_gen_Maze: the number of genera in which Maze motifs occur

top10odrs <- read_csv("top10odrs.csv", col_types = "ciii")
top10odrs

In [ ]:
dir.create("./trees")
dir.create("./statedata")
dir.create("./commands")
dir.create("./logs")
dir.create("./jobs")

In [ ]:
make_statedata_trees <- function(target_order) {
    
    df_ptn_gen <- read.csv("../annot/fish-patterns-db-gen.csv")

    tr <- fishtree_phylogeny(rank=target_order, type="chronogram")
    tr_sps <- tr$tip.label
    tr_gensps <- str_split(tr_sps, "_", simplify=TRUE)
    df_tr_gensps <- data.frame(genus = tr_gensps[, 1], species = tr_sps)
    
    # keep genera for which trait data are available
    df_tr_ptn_gensps <- df_tr_gensps %>% filter(genus %in% df_ptn_gen$genus)
    
    set.seed(42)

    tr2s <- list()

    for (i in 1:500) {
        df_tr_ptn_gensps_sample <- df_tr_ptn_gensps %>% group_by(genus) %>% sample_n(1)
        df_tr_ptn_gensps_sample
    
        # keep randomly selected species as a representative
        tr2 <- keep.tip(tr, as.character(df_tr_ptn_gensps_sample$species))
    
        # use genera names as tip.labels
        tr2$tip.label <- str_split(tr2$tip.label, "_", simplify = TRUE)[, 1]
    
        tr2 <- force.ultrametric(tr2)
    
        tr2s <- append(tr2s, list(tr2))
    }
    trees_file <- paste("./trees/", target_order, "_500.trees", sep="")
    write.nexus(tr2s, file = trees_file, translate = TRUE)

    
    # align and keep trait data for genera included in the tree
    odr <- match(tr2s[[1]]$tip.label, df_ptn_gen$genus)
    odr <- subset(odr, !is.na(odr))
    df_ptn_gen2 <- df_ptn_gen[odr, ]
    
    
    # make multistate data for triple pattern motifs
    make_multistate <- function(target_order, ptn1, ptn2, ptn3) {
        df_ms <- df_ptn_gen2[c("genus", ptn1, ptn2, ptn3)]
        df_ms <- df_ms %>% mutate(state = case_when(
            (df_ms[ptn1] == 0 & df_ms[ptn2] == 0 & df_ms[ptn3] == 0) ~ 1,
            (df_ms[ptn1] == 1 & df_ms[ptn2] == 0 & df_ms[ptn3] == 0) ~ 2,
            (df_ms[ptn1] == 0 & df_ms[ptn2] == 1 & df_ms[ptn3] == 0) ~ 3,
            (df_ms[ptn1] == 0 & df_ms[ptn2] == 0 & df_ms[ptn3] == 1) ~ 4,
            (df_ms[ptn1] == 1 & df_ms[ptn2] == 1 & df_ms[ptn3] == 0) ~ 5,
            (df_ms[ptn1] == 1 & df_ms[ptn2] == 0 & df_ms[ptn3] == 1) ~ 6,
            (df_ms[ptn1] == 0 & df_ms[ptn2] == 1 & df_ms[ptn3] == 1) ~ 7,
            (df_ms[ptn1] == 1 & df_ms[ptn2] == 1 & df_ms[ptn3] == 1) ~ 8
        ))

        state_file <- paste("./statedata/", target_order, "_state_", ptn1, "_", ptn2, "_", ptn3, ".txt", sep="")
        write.table(df_ms[c("genus", "state")],
            file = state_file,
            sep = "\t", na = "-", row.names = FALSE, col.names = FALSE, quote = FALSE)
    }

    make_multistate(target_order, "Sp_D", "Sp_L", "Maze")
    
}

In [ ]:
make_BayesTraits_commands <- function(target_order) {
    make_commands <- function(target_order, ptn1, ptn2, ptn3, rep) {
        fbase <- paste(target_order, "_state_", ptn1, "_", ptn2, "_", ptn3, sep="")
        trees_file <- paste("./trees/", target_order, "_500.trees", sep="")
        state_file <- paste("./statedata/", target_order, "_state_", ptn1, "_", ptn2, "_", ptn3, ".txt", sep="")
        
        sample <- "50000"
        iterations <- "600000000"
        burnin <- "100000000"
        hp_exp1 <- "0"
        hp_exp2 <- "10"
        cores <- "4"
        stones1 <- "100"
        stones2 <- "10000"
        
        modelpar <- paste("rjMCMC_SCT_HPexp_", hp_exp1, "_", hp_exp2, sep="")
        command_file_IND <- paste("./commands/command_", fbase, "_IND_", modelpar, "_rep_", rep, ".txt", sep="")
        command_file_DEP <- paste("./commands/command_", fbase, "_DEP_", modelpar, "_rep_", rep, ".txt", sep="")
        command_file_indZ <- paste("./commands/command_", fbase, "_indZ_", modelpar, "_rep_", rep, ".txt", sep="")
        
        command_IND_MCMC <- c("1",
                              "2",
                              paste("LogFile ", "./logs/", fbase, "_IND_", modelpar, "_rep_", rep, sep = ""),
                              paste("RJHP exp", hp_exp1, hp_exp2),
                              paste("Cores", cores),
                              "",
                              paste("Sample", sample),
                              paste("Iterations", iterations),
                              paste("BurnIn", burnin),
                              "ScaleTrees 0.1",
                              "",
                              "res q15 q16 q17 q18  0",
                              "res q23 q24 q27 q28  0",
                              "res q32 q34 q36 q38  0",
                              "res q42 q43 q45 q48  0",
                              "res q51 q54 q56 q57  0",
                              "res q61 q63 q65 q67  0",
                              "res q71 q72 q75 q76  0",
                              "res q81 q82 q83 q84  0",
                              "",
                              "res q35 q46 q78  q12",
                              "res q25 q47 q68  q13",
                              "res q26 q37 q58  q14",
                              "res q53 q64 q87  q21",
                              "res q52 q74 q86  q31",
                              "res q62 q73 q85  q41",
                              "",
                              paste("Stones", stones1, stones2),
                              "",
                              "Run")
        write(command_IND_MCMC, file = command_file_IND)

        command_indZ_MCMC <- c("1",
                               "2",
                               paste("LogFile ", "./logs/", fbase, "_indZ_", modelpar, "_rep_", rep, sep = ""),
                               paste("RJHP exp", hp_exp1, hp_exp2),
                               paste("Cores", cores),
                               "",
                               paste("Sample", sample),
                               paste("Iterations", iterations),
                               paste("BurnIn", burnin),
                               "ScaleTrees 0.1",
                               "",
                               "res q15 q16 q17 q18  0",
                               "res q23 q24 q27 q28  0",
                               "res q32 q34 q36 q38  0",
                               "res q42 q43 q45 q48  0",
                               "res q51 q54 q56 q57  0",
                               "res q61 q63 q65 q67  0",
                               "res q71 q72 q75 q76  0",
                               "res q81 q82 q83 q84  0",
                               "",
                               "res q26 q37 q58  q14",
                               "res q62 q73 q85  q41",
                               "res q78 q35",
                               "res q87 q53",
                               "res q46 q12",
                               "res q64 q21",
                               "res q47 q13",
                               "res q74 q31",
                               "res q68 q25",
                               "res q86 q52",
                               "",
                               paste("Stones", stones1, stones2),
                               "",
                               "Run")
        write(command_indZ_MCMC, file = command_file_indZ)

        command_DEP_MCMC <- c("1",
                              "2",
                              paste("LogFile ", "./logs/", fbase, "_DEP_", modelpar, "_rep_", rep, sep = ""),
                              paste("RJHP exp", hp_exp1, hp_exp2),
                              paste("Cores", cores),
                              "",
                              paste("Sample", sample),
                              paste("Iterations", iterations),
                              paste("BurnIn", burnin),
                              "ScaleTrees 0.1",
                              "",
                              "res q15 q16 q17 q18  0",
                              "res q23 q24 q27 q28  0",
                              "res q32 q34 q36 q38  0",
                              "res q42 q43 q45 q48  0",
                              "res q51 q54 q56 q57  0",
                              "res q61 q63 q65 q67  0",
                              "res q71 q72 q75 q76  0",
                              "res q81 q82 q83 q84  0",
                              "",
                              paste("Stones", stones1, stones2),
                              "",
                              "Run")
        write(command_DEP_MCMC, file = command_file_DEP)
        
        job_file <- paste(fbase, "_", modelpar, "_rep_", rep, ".job", sep="")
        job_contents <- c("#!/bin/bash",
                          "#$ -S /bin/bash",
                          "#$ -cwd",
                          paste("#$ -pe smp", cores),
                          "",
                          "export LD_LIBRARY_PATH=$HOME/lcl/lib64:$HOME/lcl/lib:$LD_LIBRARY_PATH",
                          "export PATH=$HOME/bin:$HOME/lcl/bin:$PATH",
                          "n=$SGE_TASK_ID",
                          "",
                          "cd ..",
                          "pwd",
                          "",
                          "date",
                          ". job_start_slackpost.sh",
                          "",
                          "case ${n} in",
                          "    1) ",
                          paste("    job_slackpost.sh '", fbase, " [IND] start'", sep=""),
                          paste("    ./BayesTraitsV3 ", trees_file, " ", state_file, " < ", command_file_IND, ";;", sep = ""),
                          "",
                          "    2) ",
                          paste("    job_slackpost.sh '", fbase, " [indZ] start'", sep=""),
                          paste("    ./BayesTraitsV3 ", trees_file, " ", state_file, " < ", command_file_indZ, ";;", sep = ""),
                          "",
                          "    3) ",
                          paste("    job_slackpost.sh '", fbase, " [DEP] start'", sep=""),
                          paste("    ./BayesTraitsV3 ", trees_file, " ", state_file, " < ", command_file_DEP, ";;", sep = ""),
                          "",
                          "esac",
                          "",
                          ". job_end_slackpost.sh")
        write(job_contents, file = paste("./jobs/", job_file, sep=""))
        
        return(job_file)

    }

    job_files <- list()
    job_files <- c(job_files, make_commands(target_order, "Sp_D", "Sp_L", "Maze", 1))
    job_files <- c(job_files, make_commands(target_order, "Sp_D", "Sp_L", "Maze", 2))
    job_files <- c(job_files, make_commands(target_order, "Sp_D", "Sp_L", "Maze", 3))

    return(job_files)
}

In [ ]:
for (target_order in top10odrs[[1]]) {
    print(target_order)
    make_statedata_trees(target_order)
    job_files <- make_BayesTraits_commands(target_order)
    write_lines(job_files, paste("./jobs/", target_order, "_jobs.txt", sep=""))
}